In [ ]:
from httpx import Client
import polars as pl
from concurrent.futures import ThreadPoolExecutor

In [ ]:
client = Client(base_url="https://fantasy.premierleague.com/api", event_hooks={"response": [lambda x: x.raise_for_status()]})

In [ ]:
def get_entry_picks(entry_id: int, gameweek_id: int) -> pl.DataFrame:
    """
    Returns the players picked for a gameweek
    """
    
    df = (pl.DataFrame(client.get(f"entry/{entry_id}/event/{gameweek_id}/picks/").json()["picks"])
          .with_columns(entry_id=entry_id)
          .rename({"element": "player_id"})
          )

    return df

def get_entry_history(entry_id: int) -> pl.DataFrame:
    """
    Returns the points by week for the entry
    """
    
    df = (pl.DataFrame(client.get(f"entry/{entry_id}/history/").json()["current"])
          .with_columns(entry_id=entry_id)
          .rename({"event": "gameweek_id"})
          .select(["gameweek_id", "entry_id", "total_points"])
          )

    return df

In [ ]:
bootstrap_data = client.get("bootstrap-static/").json()

##### player/team static data

In [ ]:
gameweeks_df = (pl.DataFrame(bootstrap_data["events"])
                .rename({"id": "gameweek_id"})
                .select(["gameweek_id", "deadline_time", "is_current"])
                )

gameweek_id = gameweeks_df.filter(pl.col("is_current")).row(0, named=True)["gameweek_id"]

players_df = (pl.json_normalize(bootstrap_data["elements"])
              .select(["id", "web_name", "team", "element_type"])
              .rename({"id": "player_id", "team": "team_id", "element_type": "position_id"})
              )

positions_df = (pl.json_normalize(bootstrap_data["element_types"])
                .select(["id", "singular_name"])
                .rename({"id": "position_id", "singular_name": "position_name"})
                )

teams_df = (pl.json_normalize(bootstrap_data["teams"])
            .select(["id", "name"])
            .rename({"id": "team_id", "name": "team_name"})
            )

players_df = players_df.join(teams_df, on="team_id", how="left").join(positions_df, on="position_id")

##### league entry data

In [ ]:
league_df = (pl.DataFrame(client.get("leagues-classic/737576/standings/").json()["standings"]["results"])
             .rename({"entry": "entry_id", "player_name": "manager_name"})
             .select(["entry_id", "manager_name", "entry_name", "total"])
             .with_columns(pl.col("entry_id").cast(pl.Int32))
             )

# get current gameweek picks for each entry
with ThreadPoolExecutor() as executor:
    picks = list(executor.map(lambda entry_id: get_entry_picks(
        entry_id, gameweek_id), league_df["entry_id"].to_list()))
    
# get weekly points for each entry
with ThreadPoolExecutor() as executor:
    history = list(executor.map(lambda entry_id: get_entry_history(entry_id), league_df["entry_id"].to_list()))

entry_picks_df = (pl.concat(picks)
                  .join(players_df, on="player_id")
                  .join(league_df, on="entry_id")
                  .select(["entry_id", "manager_name", "web_name", "team_name", "position_name", "player_id", "multiplier", "is_captain", "position"])
                  )

history = (pl.concat(history)
                  .join(league_df, on="entry_id")
                  .select(["gameweek_id", "entry_id", "manager_name", "total_points"])
                  )

In [ ]:
history.pivot(values="total_points", on="gameweek_id", aggregate_function="first").sort("manager_name")